<a href="https://colab.research.google.com/github/singhvertika119/Learning-DL-Pytorch/blob/main/digitClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [16]:
# check availability of GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

using device: cuda


In [17]:
# transform image to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [18]:
# download dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [19]:
# create loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [20]:
# Define neural network
class SimpleNN(nn.Module):
  def __init__(self):
    super(SimpleNN, self).__init__()
    self.fc1 = nn.Linear(28*28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [21]:
# initialize model, loss function, optimizer
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
# save model
torch.save(model.state_dict(), "mnist_model.pth")
print("Model saved successfully!")
loaded_model = SimpleNN()
loaded_model.load_state_dict(torch.load("mnist_model.pth"))
loaded_model.eval()
print("model loaded successfully")

Model saved successfully!
model loaded successfully


In [23]:
# training loop
epochs = 5
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/5, Loss: 0.3908
Epoch 2/5, Loss: 0.1889
Epoch 3/5, Loss: 0.1367
Epoch 4/5, Loss: 0.1111
Epoch 5/5, Loss: 0.0945


In [24]:
# testing the model
model.eval()
correct, total = 0, 0
with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 96.95%


In [27]:
# test on a random sample
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# dummy tensor for testing
dummy_input = torch.rand(1, 1, 28, 28)
output = loaded_model(dummy_input)
print("Model ouput:", output)
predicted_digit = torch.argmax(output, dim=1).item()
print(f"Predicted digit: {predicted_digit}")

Model ouput: tensor([[-0.1473,  0.2014,  0.0161,  0.2040,  0.0110, -0.1377,  0.0029,  0.0758,
          0.0972, -0.0216]], grad_fn=<AddmmBackward0>)
Predicted digit: 3
